In [90]:
import warnings
warnings.filterwarnings('ignore')

In [124]:
import pandas as pd
import numpy as np

### Importamos y leemos nuestro archivo, vemos el nombre de las hojas

In [199]:
data = pd.ExcelFile('data/BASE DATOS  JULIO - 2013 - ESCLEROSIS MULTIPLE V3.xlsx')

data.sheet_names

['Diagnóstico Diferencial-Hoja3',
 'FORMAS PRIMARIAS Hoja2',
 'Datos por fármaco y mes',
 'MIELOPATIAS',
 'ENFERMEDAD DE DEVIC',
 'List todos los pacientes-Hoja1',
 'Hoja1']

### La hoja que nos interesa es la de 'List todos los pacientes-Hoja1' y es la que asignaremos como dataframe a 'pacientes_em'

In [200]:
pacientes_em = pd.read_excel('data/BASE DATOS  JULIO - 2013 - ESCLEROSIS MULTIPLE V3.xlsx', 5)

pacientes_em.head()

,Unnamed: 0,EDAD,SEXO,Unnamed: 3,TRATAMIENTO,Unnamed: 5,CENTRO PILOTO,INICIO TTO.
0,NOMBRE DEL PACIENTE,EDAD,SEXO,CEDULA,TRATAMIENTO,AMPYRA,CIUDAD,INICIO TTO.
1,NaN,45,M,NaN,REBIF,NaN,CARACAS,2002-09-01 00:00:00
2,NaN,43,F,NaN,BETAFERON,NaN,PTO. LA CRUZ,2001-06-01 00:00:00
3,NaN,39,F,NaN,BETAFERON,NaN,GUACARA,2010-08-01 00:00:00
4,NaN,35,F,NaN,AVONEX,NaN,CARACAS,2004-07-01 00:00:00


In [201]:
pacientes_em.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2099 entries, 0 to 2098
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     1 non-null      object
 1   EDAD           2085 non-null   object
 2   SEXO           2058 non-null   object
 3   Unnamed: 3     1 non-null      object
 4   TRATAMIENTO    2095 non-null   object
 5   Unnamed: 5     2 non-null      object
 6   CENTRO PILOTO  2094 non-null   object
 7   INICIO TTO.    2085 non-null   object
dtypes: object(8)
memory usage: 131.3+ KB


### Renombramos la columna 'CENTRO PILOTO' como 'CIUDAD'

In [202]:
pacientes_em.rename(columns={'CENTRO PILOTO': 'CIUDAD'}, inplace=True)

### Eliminamos las columnas 'Unnamed: 0','Unnamed: 3','Unnamed: 5' que no contienen información relevante

In [203]:
pacientes_em = pacientes_em.drop(columns=['Unnamed: 0','Unnamed: 3','Unnamed: 5'])

### Eliminamos la primera fila que es la repetición del nombre de las columnas

In [204]:
pacientes_em = pacientes_em.drop(pacientes_em.index[0])
pacientes_em = pacientes_em.reset_index(drop=True)

### Chequeamos los valores nulos

In [205]:
pacientes_em.isnull().sum()

EDAD           14
SEXO           41
TRATAMIENTO     4
CIUDAD          5
INICIO TTO.    14
dtype: int64

### Comenzaremos explorando y limpiando la columna 'TRATAMIENTO' ya que es la mas importante en mi análisis

In [206]:
pacientes_em['TRATAMIENTO'].unique()

array(['REBIF', 'BETAFERON', 'AVONEX', 'TYSABRI', 'GILENYA', 'COPAXONE',
       'NINGUNO', 'TECFIDERA', 'AUBAGIO', nan, 'AVONEX ', 'AZATIOPRINA',
       'NO EM', 'LEMTRADA', 'OCREVUS'], dtype=object)

### Eliminamos las filas con valores nulos en la columna de tratamiento, ya que es un dato fundamental para mi estudio y no me sirve desconocer el tratamiento

In [207]:
pacientes_em[pacientes_em['TRATAMIENTO'].isnull()].index

Int64Index([475, 1984, 1987, 2016], dtype='int64')

In [208]:
pacientes_em.drop(index = [475, 1984, 1987, 2016], inplace = True)

### Eliminamos los tratamientos 'TECFIDERA', 'AUBAGIO' y 'AZATIOPRINA' ya que no eran tratamientos que suminitrara la Seguridad Social en Venezuela

In [209]:
valores_a_eliminar = ['TECFIDERA', 'AUBAGIO', 'AZATIOPRINA']
pacientes_em = pacientes_em[~pacientes_em['TRATAMIENTO'].isin(valores_a_eliminar)]

In [210]:
pacientes_em['TRATAMIENTO'].value_counts()

REBIF        679
AVONEX       576
BETAFERON    446
COPAXONE     206
TYSABRI      137
GILENYA       23
NINGUNO        9
LEMTRADA       5
OCREVUS        5
AVONEX         3
NO EM          1
Name: TRATAMIENTO, dtype: int64

### Además, eliminamos los pacientes que no tienen ningún tratamiento y el paciente que no tiene EM

In [211]:
valores_a_eliminar = ['NINGUNO', 'NO EM']
pacientes_em = pacientes_em[~pacientes_em['TRATAMIENTO'].isin(valores_a_eliminar)]

In [212]:
pacientes_em['TRATAMIENTO'] = pacientes_em['TRATAMIENTO'].replace({'AVONEX ': 'AVONEX'})

In [213]:
pacientes_em['TRATAMIENTO'].value_counts()

REBIF        679
AVONEX       579
BETAFERON    446
COPAXONE     206
TYSABRI      137
GILENYA       23
LEMTRADA       5
OCREVUS        5
Name: TRATAMIENTO, dtype: int64

In [214]:
pacientes_em.isnull().sum()

EDAD           13
SEXO           37
TRATAMIENTO     0
CIUDAD          3
INICIO TTO.    11
dtype: int64

### Vamos a explorar y revisar la columna 'EDAD'

In [215]:
pacientes_em['EDAD'].unique()

array([45, 43, 39, 35, 46, 12, 8, 29, 21, 13, 26, 30, 53, 33, 42, 31, 48,
       16, 25, 36, 34, 24, 51, 50, 23, 44, 54, 65, 28, 38, 14, 41, 17, 55,
       19, 40, 32, 61, 52, 37, 7, 27, 56, 22, 5, 62, 66, 49, 9, 18, 58,
       47, 72, 57, 71, 59, 6, 15, 63, 60, 64, 20, 73, 10, 67, 79, 4, 11,
       69, nan], dtype=object)

In [216]:
# Convertir la columna 'EDAD' a tipo entero
pacientes_em['EDAD'] = pacientes_em['EDAD'].replace('unknown', np.nan).astype(float).astype('Int64')

### Continuamos explorando la columna 'CIUDAD', vemos que hay valores escritos de distintas formas para las mismas ciudades, por lo que limpiaremos esas datos

In [217]:
pacientes_em['CIUDAD'].unique()

array(['CARACAS', 'PTO. LA CRUZ', 'GUACARA', 'VALENCIA', 'MARACAY',
       'MARACAIBO', 'BARINAS', 'CORO', 'LA VICTORIA', 'BARQUISIMETO',
       'PORLAMAR', 'SAN CRISTOBAL', 'PTO.ORDAZ', 'BARCELONA', 'CUMANA',
       'SAN FELIPE', 'CIUDAD BOLIVAR', 'MERIDA', 'MATURIN', 'CHARALLAVE',
       'CABIMAS', 'PTO. FIJO', 'SAN FERNANDO', 'TRUJILLO',
       'OCUMARE DEL TUY', 'SAN JUAN', 'UPATA', 'EL TIGRE', 'LOS TEQUES',
       'CARUPANO', 'ANACO', 'LA GUAIRA', 'VALERA', 'SAN JUAN ',
       'ACARIGUA', 'SAN ANTONIO', 'PTO. ORDAZ', 'TUCUPITA', 'EL VIGIA',
       'YAGUA', 'PTO- FIJO', 'SAN CARLOS', 'EL TIGRITO', 'GUARENAS',
       'PUNTO FIJO', 'HDL', 'TURMERO', 'CANTAURA', 'GUANARE', 'PTO FIJO',
       'VALLE DE LA PASCUA', 'CAGUA', 'LOS GUAYOS', 'CALABOZO',
       'STA. TERESA DEL TUY', 'TINAQUILLO', nan, 'MARCAY', 'GUATIRE',
       'PTO.LA CRUZ', 'SAN FELIX', 'PAMPATAR', 'CAICARA DEL ORINOCO',
       'TINACO', 'SAN JUAN DE LOS MORROS'], dtype=object)

In [218]:
pacientes_em['CIUDAD'] = pacientes_em['CIUDAD'].replace(
                                        {'PTO. LA CRUZ': 'PUERTO LA CRUZ', 'PTO.LA CRUZ': 'PUERTO LA CRUZ',
                                        'PTO FIJO': 'PUNTO FIJO','PTO. FIJO': 'PUNTO FIJO',
                                        'PTO- FIJO': 'PUNTO FIJO', 'CIACARA DEL ORINOCO': 'CAUCARA DEL ORINOCO', 
                                        'PTO.ORDAZ':'PUERTO ORDAZ','PTO. ORDAZ' : 'PUERTO ORDAZ',
                                        'SAN JUAN ': 'SAN JUAN', 'MARCAY':'MARACAY', 'STA. TERESA DEL TUY': 'SANTA TERESA DEL TUY'})

In [219]:
pacientes_em['CIUDAD'].value_counts()

CARACAS                   907
MARACAIBO                 256
VALENCIA                  139
BARQUISIMETO              111
MARACAY                   106
SAN CRISTOBAL              71
PUERTO ORDAZ               57
PUERTO LA CRUZ             54
PUNTO FIJO                 47
MERIDA                     40
MATURIN                    37
CIUDAD BOLIVAR             29
PORLAMAR                   27
LOS TEQUES                 23
CORO                       17
SAN JUAN                   14
LA VICTORIA                13
TRUJILLO                   12
BARCELONA                  10
BARINAS                    10
ACARIGUA                    8
LA GUAIRA                   8
CUMANA                      7
SAN FERNANDO                7
EL TIGRE                    6
SAN FELIPE                  6
GUANARE                     4
VALERA                      4
GUARENAS                    3
CAGUA                       3
SAN CARLOS                  3
EL VIGIA                    2
CANTAURA                    2
TURMERO   

### Asignaremos 'Unknow' a los valores nulos que quedan, no los elimino porque contienen nombres de tratamientos que son fundamentales en mi análisis

In [220]:
pacientes_em['SEXO'] = pacientes_em['SEXO'].fillna('unknown')

In [221]:
pacientes_em['CIUDAD'] = pacientes_em['CIUDAD'].fillna('unknown')

In [222]:
pacientes_em['INICIO TTO.'] = pacientes_em['INICIO TTO.'].fillna('unknown')

### Agregamos una columna nueva llamada 'PRECIO TRATAMIENTO USD' a la cual según el nombre del tratamiento, le asignaremos el costo del mismo

In [223]:
precios_tratamientos = {'AVONEX': 18000,
                        'BETAFERON': 26000,
                        'REBIF': 24000,
                        'COPAXONE': 14000,
                        'TYSABRI': 36000,
                        'GILENYA': 32000,
                        'LEMTRADA': 80000,
                        'OCREVUS': 24000
                        }

In [224]:
pacientes_em['PRECIO TRATAMIENTO ($)'] = pacientes_em['TRATAMIENTO'].map(precios_tratamientos)

In [225]:
pacientes_em.head()

,EDAD,SEXO,TRATAMIENTO,CIUDAD,INICIO TTO.,PRECIO TRATAMIENTO ($)
0,45,M,REBIF,CARACAS,2002-09-01 00:00:00,24000
1,43,F,BETAFERON,PUERTO LA CRUZ,2001-06-01 00:00:00,26000
2,39,F,BETAFERON,GUACARA,2010-08-01 00:00:00,26000
3,35,F,AVONEX,CARACAS,2004-07-01 00:00:00,18000
4,46,M,TYSABRI,VALENCIA,2007-04-01 00:00:00,36000


In [226]:
pacientes_em.reset_index(drop=True, inplace=True)

In [227]:
pacientes_em['PRECIO TRATAMIENTO ($)'].sum()

47386000

### Agregamos la columna 'Estado' para complementar la información

In [228]:
estados = {'CARACAS': 'Estado Distrito Capital', 'PUERTO LA CRUZ': 'Estado Anzoátegui', 'GUACARA': 'Estado Carabobo',
                      'VALENCIA': 'Estado Carabobo', 'MARACAY': 'Estado Aragua', 'MARACAIBO': 'Estado Zulia',
                      'BARINAS': 'Estado Barinas', 'CORO': 'Estado Falcón', 'LA VICTORIA': 'Estado Aragua',
                      'BARQUISIMETO': 'Estado Lara', 'PORLAMAR': 'Estado Nueva Esparta', 'SAN CRISTOBAL': 'Estado Táchira',
                      'PUERTO ORDAZ': 'Estado Bolívar', 'BARCELONA': 'Estado Anzoátegui', 'CUMANA': 'Estado Sucre',
                      'SAN FELIPE': 'Estado Yaracuy', 'CIUDAD BOLIVAR': 'Estado Bolívar', 'MERIDA': 'Estado Mérida',
                      'MATURIN': 'Estado Monagas', 'CHARALLAVE': 'Estado Miranda', 'CABIMAS': 'Estado Zulia',
                      'PUNTO FIJO': 'Estado Falcón', 'SAN FERNANDO': 'Estado Apure', 'TRUJILLO': 'Estado Trujillo',
                      'OCUMARE DEL TUY': 'Estado Miranda', 'SAN JUAN': 'Estado Guárico', 'UPATA': 'Estado Bolívar',
                      'EL TIGRE': 'Estado Anzoátegui', 'LOS TEQUES': 'Estado Miranda', 'CARUPANO': 'Estado Sucre',
                      'ANACO': 'Estado Anzoátegui', 'LA GUAIRA': 'Estado La Guaira', 'VALERA': 'Estado Trujillo',
                      'ACARIGUA': 'Estado Portuguesa', 'SAN ANTONIO': 'Estado Táchira', 'TUCUPITA': 'Estado Delta Amacuro',
                      'EL VIGIA': 'Estado Mérida', 'YAGUA': 'Estado Carabobo', 'SAN CARLOS': 'Estado Cojedes',
                      'EL TIGRITO': 'Estado Anzoátegui', 'GUARENAS': 'Estado Miranda', 'TURMERO': 'Estado Aragua',
                      'CANTAURA': 'Estado Anzoátegui', 'GUANARE': 'Estado Portuguesa', 'VALLE DE LA PASCUA': 'Estado Guárico',
                      'CAGUA': 'Estado Aragua', 'LOS GUAYOS':'Estado Carabobo', 'CALABOZO': 'Estado Guárico',
                      'SANTA TERESA DEL TUY': 'Estado Miranda', 'TINAQUILLO': 'Estado Cojedes', 'GUATIRE': 'Estado Miranda',
                      'SAN FELIX': 'Estado Bolívar', 'PAMPATAR': 'Estado Nueva Esparta', 'CAICARA DEL ORINOCO': 'Estado Bolívar',
                      'TINACO': 'Estado Cojedes', 'SAN JUAN DE LOS MORROS': 'Estado Guárico'
                      }

In [229]:
pacientes_em.insert(4, 'ESTADO', pacientes_em['CIUDAD'].map(estados))

In [230]:
pacientes_em

,EDAD,SEXO,TRATAMIENTO,CIUDAD,ESTADO,INICIO TTO.,PRECIO TRATAMIENTO ($)
0,45,M,REBIF,CARACAS,Estado Distrito Capital,2002-09-01 00:00:00,24000
1,43,F,BETAFERON,PUERTO LA CRUZ,Estado Anzoátegui,2001-06-01 00:00:00,26000
2,39,F,BETAFERON,GUACARA,Estado Carabobo,2010-08-01 00:00:00,26000
3,35,F,AVONEX,CARACAS,Estado Distrito Capital,2004-07-01 00:00:00,18000
4,46,M,TYSABRI,VALENCIA,Estado Carabobo,2007-04-01 00:00:00,36000
...,...,...,...,...,...,...,...
2075,<NA>,F,OCREVUS,LA GUAIRA,Estado La Guaira,2019-05-19 00:00:00,24000
2076,<NA>,M,OCREVUS,MARACAIBO,Estado Zulia,2019-02-01 00:00:00,24000
2077,<NA>,M,OCREVUS,CARACAS,Estado Distrito Capital,2020-03-20 00:00:00,24000
2078,<NA>,F,OCREVUS,CARACAS,Estado Distrito Capital,2021-06-21 00:00:00,24000


### Agregaremos una nueva columna llamada 'AÑO' que será útil para calcular el total de costo de los tratamientos médicos por año

In [236]:
#Aplicamos la función extraer_año para según la fecha de inicio de tratamiento, nos agregue el año en la nueva columna
pacientes_em['AÑO'] = pacientes_em['INICIO TTO.'].apply(lambda x: extraer_año(x) )

In [237]:
pacientes_em

,EDAD,SEXO,TRATAMIENTO,CIUDAD,ESTADO,INICIO TTO.,PRECIO TRATAMIENTO ($),AÑO
0,45,M,REBIF,CARACAS,Estado Distrito Capital,2002-09-01 00:00:00,24000,2002
1,43,F,BETAFERON,PUERTO LA CRUZ,Estado Anzoátegui,2001-06-01 00:00:00,26000,2001
2,39,F,BETAFERON,GUACARA,Estado Carabobo,2010-08-01 00:00:00,26000,2010
3,35,F,AVONEX,CARACAS,Estado Distrito Capital,2004-07-01 00:00:00,18000,2004
4,46,M,TYSABRI,VALENCIA,Estado Carabobo,2007-04-01 00:00:00,36000,2007
...,...,...,...,...,...,...,...,...
2075,<NA>,F,OCREVUS,LA GUAIRA,Estado La Guaira,2019-05-19 00:00:00,24000,2019
2076,<NA>,M,OCREVUS,MARACAIBO,Estado Zulia,2019-02-01 00:00:00,24000,2019
2077,<NA>,M,OCREVUS,CARACAS,Estado Distrito Capital,2020-03-20 00:00:00,24000,2020
2078,<NA>,F,OCREVUS,CARACAS,Estado Distrito Capital,2021-06-21 00:00:00,24000,2021


In [240]:
# Luego, agrupamos por año para saber el costo en tratamientos por año
pacientes_em.groupby(pacientes_em['AÑO'])['PRECIO TRATAMIENTO ($)'].sum()

AÑO
1900     436000
1992      18000
1993      26000
1994      48000
1995      74000
1996     174000
1997     106000
1998      98000
1999    1620000
2000    3200000
2001    3064000
2002    2536000
2003    2916000
2004    3420000
2005    4134000
2006    3364000
2007    4356000
2008    4198000
2009    3176000
2010    3102000
2011    2592000
2012    2474000
2013    1960000
2017      80000
2018      80000
2019      48000
2020      24000
2021      24000
2022      24000
2023      14000
Name: PRECIO TRATAMIENTO ($), dtype: int64

In [242]:
años = [2005, 2007, 2009, 2010, 2013]
filtro_años = pacientes_em[pacientes_em['AÑO'].isin(años)]

# Sumar la columna 'PRECIO TRATAMIENTO ($)'
filtro_años.groupby('AÑO')['PRECIO TRATAMIENTO ($)'].sum()

AÑO
2005    4134000
2007    4356000
2009    3176000
2010    3102000
2013    1960000
Name: PRECIO TRATAMIENTO ($), dtype: int64

In [244]:
tratamiento = filtro_años.groupby('AÑO')['PRECIO TRATAMIENTO ($)'].sum().sum()
construc = 78339000000
tratamiento

16728000

In [245]:
tratamiento = 1960000

In [246]:
total_pastaca = tratamiento+construc
porcenta_tratamiento = tratamiento/total_pastaca*100
porcenta_tratamiento

0.0025018840718827036

### Exportamos nuestro archivo

In [247]:
pacientes_em.to_excel('pacientes_em.xlsx', index=False)